In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os

import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cdist

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
#Importación de archivos de afiliados
inicio = time.time()
afi = pd.read_csv( ruta_afi, delimiter='\t', encoding='iso-8859-1', decimal='.')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 57.05829310417175  segundos


In [4]:
#Agrupamiento por relación de trabajo
inicio = time.time()
agrupa = [ (afi['SECTOR'].isin(['PRIVADO', 'PASANTE PRIVADO'])), 
           (afi['SECTOR'].isin(['PUBLICO', 'PASANTE PUBLICO'])), 
           (afi['SECTOR'].isin(['INDEPENDIENTES', '108-ARTISTA Y GESTOR DE CULTURA', 
                                '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  '])),
           (afi['SECTOR'].isin(['69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
                                '89-AFILIACION DOMICILIADO EN EL EXTERIOR'])),
           (afi['SECTOR'] == '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA'),
           (afi['SECTOR'] == '06-CODIGO DEL TRABAJO - CT ')
            ]
            
tipo = ['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR']

afi['SECTOR_A'] = np.select( agrupa, tipo, default=afi['SECTOR'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 20.642767429351807  segundos


In [5]:
afi['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       'INDEPENDIENTES',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       'PASANTE PRIVADO',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR', 'PASANTE PUBLICO',
       '108-ARTISTA Y GESTOR DE CULTURA', '06-CODIGO DEL TRABAJO - CT '],
      dtype=object)

In [6]:
afi['SECTOR_A'].unique()

array(['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR'], dtype=object)

In [7]:
afi.shape

(68062270, 13)

In [8]:
#Para acceder a las prestaciones de IVM debe tener al menos 5 años de aporte o 60 imposiciones
ced_counts = afi.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()

In [9]:
# numero de cedulas que tienen menos de 60 imposiciones
ced_counts[ced_counts < 60].index.nunique()

114212

In [10]:
#Se filtran a las cedulas con menos de 60 imposicones 
inicio = time.time()
afi_fil = afi[~afi['CEDULA_COD'].isin(ced_counts[ced_counts < 60].index)].copy()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se crea una variable fecha
inicio = time.time()
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se ordenan los registros
inicio = time.time()
afi_fil = afi_fil.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
afi_fil.shape

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 24.8166286945343  segundos
Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 28.197264909744263  segundos
Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 41.15096068382263  segundos


(64638969, 14)

In [11]:
# Especifica la ruta del directorio donde quieres guardar el archivo
directorio = r_ruta

# Asegúrate de que el directorio existe
os.makedirs(directorio, exist_ok=True)

# Nombre del archivo
nombre_archivo = 'viu_clean_afiliados.pkl'

# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)

# Objetos a guardar
objeto1 = afi
objeto2 = afi_fil

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto1, archivo)
    pickle.dump(objeto2, archivo)
    
# #Cargar los archivos
# with open( ruta_archivo, 'rb') as archivo:
#     pickle.load(archivo)  # Carga y descarta el primer objeto
#     afi_fil = pickle.load(archivo)  # Carga el segundo objeto    

In [12]:
#Se eliminan archivos no necesarios para liberar la memoria
del afi
del objeto1
del objeto2
del ced_counts

In [13]:
afi_fil #Tiene una dimensión de (64638969 rows × 14 columns)

,CEDULA_COD,ANIO,MES,CODSEC,SECTOR,DESTIPEMP,DESRELTRA,CODCII,PROVINCIA,CANTON,SALARIO,PORCENTAJE_AP,SECTOR_A,FECHA
26481585,126,2004,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,147.45,20.5,PRI,2004-12-01
67442511,126,2005,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,41.0,PRI,2005-01-01
41179435,126,2005,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-02-01
44033860,126,2005,3,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-03-01
47099991,126,2005,4,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896256,23594465,2017,11,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14045.72,20.6,PRI,2017-11-01
5107034,23594465,2017,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14164.25,20.6,PRI,2017-12-01
44333802,23594465,2018,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,12903.21,20.6,PRI,2018-01-01
38102314,23594465,2018,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,42432.21,20.6,PRI,2018-02-01


In [14]:
#Se seleccionan las características de la población para los año y mes de aportación
afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES', 'FECHA','SALARIO', 'SECTOR_A']].copy()
del afi_fil

In [15]:
#Inicio de la HL
caract = afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index() 
caract.rename( columns={'FECHA': 'INI_HL'}, inplace=True)
#Fin de la HL
caract['FIN_HL'] = caract['CEDULA_COD'].map( afi_sel.groupby('CEDULA_COD')['FECHA'].max() )
#Contar los meses, considerando duplicidad en los meses que puede tener aporte simultaneos
caract['MES_AS'] = caract['CEDULA_COD'].map( afi_sel['CEDULA_COD'].value_counts() )
#Meses trabajados unicos
aux = afi_sel.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])
caract['MES_TU'] = caract['CEDULA_COD'].map( aux['CEDULA_COD'].value_counts() )

In [16]:
objeto3 = caract

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto3, archivo)

In [17]:
del objeto3

In [18]:
caract

,CEDULA_COD,INI_HL,FIN_HL,MES_AS,MES_TU
0,126,2004-12-01,2010-12-01,73,73
1,152,2004-07-01,2009-09-01,63,63
2,216,2004-07-01,2009-09-01,98,63
3,230,2004-12-01,2015-05-01,126,126
4,235,2004-08-01,2011-06-01,83,83
...,...,...,...,...,...
442565,23075971,2017-02-01,2022-10-01,66,66
442566,23402574,2010-01-01,2019-02-01,110,110
442567,23456125,2016-07-01,2022-03-01,69,69
442568,23503686,2004-09-01,2011-11-01,87,87


In [19]:
#Personas que tienen varios aportes simultaneos
caract[caract['MES_AS']>caract['MES_TU']]

,CEDULA_COD,INI_HL,FIN_HL,MES_AS,MES_TU
2,216,2004-07-01,2009-09-01,98,63
10,397,2004-07-01,2012-01-01,138,91
19,461,2004-07-01,2010-09-01,221,75
22,472,2004-09-01,2012-01-01,176,89
28,506,2004-06-01,2010-09-01,149,76
...,...,...,...,...,...
442555,20949683,2011-08-01,2019-07-01,92,91
442557,21035240,2010-03-01,2019-09-01,112,111
442558,21035907,2006-07-01,2013-08-01,68,67
442559,21104059,2014-07-01,2023-02-01,105,104


In [20]:
#afi_sel[afi_sel['CEDULA_COD']==21729781].tail(60)

In [21]:
afi_sel =  afi_sel.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel['NUM_SEC_MES'] = 1
afi_sel['%_NUM_SECTOR'] = afi_sel['SALARIO']

In [22]:
aux = afi_sel[afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]
aux1 = afi_sel[~afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]

In [23]:
#Se debe analizar a las personas que tienen varios aportes en un mismo mes
#Se suman los salarios y se concatenan los sectores para la cedula que en el mismo anio y mes tienen más de un sector
inicio = time.time()
afi_sel_g = aux.groupby(['CEDULA_COD', 'ANIO', 'MES']).agg({'SALARIO': 'sum',
                                                            'SECTOR_A': lambda x: ';'.join(x),
                                                            'NUM_SEC_MES': lambda x: len(x),
                                                            '%_NUM_SECTOR': lambda x:  ';'.join((x / x.sum()).round(4).astype(str))
                                                                }).reset_index()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


Tiempo de ejecución es:  0.0  horas con  9.0  minutos y 3.7840187549591064  segundos


In [24]:
afi_sel_g

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR
0,216,2004,9,451.63,PUB;PRI,2,0.6265;0.3735
1,216,2004,10,451.63,PUB;PRI,2,0.6265;0.3735
2,216,2004,11,451.63,PRI;PUB,2,0.3735;0.6265
3,216,2004,12,451.63,PUB;PRI,2,0.6265;0.3735
4,216,2005,1,487.94,PUB;PRI,2,0.6379;0.3621
...,...,...,...,...,...,...,...
2347227,20949683,2014,3,696.22,PRI;PUB,2,0.7597;0.2403
2347228,21035240,2010,9,332.59,PRI;PRI,2,0.5845;0.4155
2347229,21035907,2008,6,409.27,PRI;PUB,2,0.4154;0.5846
2347230,21104059,2016,4,270.72,PRI;PRI,2,0.4459;0.5541


In [25]:
afi_sel_g_all = pd.concat( [aux1[['CEDULA_COD', 'ANIO', 'MES', 'SALARIO', 'SECTOR_A', 'NUM_SEC_MES', '%_NUM_SECTOR']], 
                            afi_sel_g], ignore_index=True )

In [26]:
afi_sel_g_all

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR
0,126,2010,12,8.06,PRI,1,8.06
1,126,2010,11,250.00,PRI,1,250.0
2,126,2010,10,250.00,PRI,1,250.0
3,126,2010,9,250.00,PRI,1,250.0
4,126,2010,8,250.00,PRI,1,250.0
...,...,...,...,...,...,...,...
62130162,20949683,2014,3,696.22,PRI;PUB,2,0.7597;0.2403
62130163,21035240,2010,9,332.59,PRI;PRI,2,0.5845;0.4155
62130164,21035907,2008,6,409.27,PRI;PUB,2,0.4154;0.5846
62130165,21104059,2016,4,270.72,PRI;PRI,2,0.4459;0.5541


In [27]:
objeto4 = afi_sel_g_all

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto4, archivo)

In [28]:
#afi_sel_g_all = objetos[3]

In [29]:
del aux
del aux1
del objeto4

In [30]:
afi_sel_g_all = afi_sel_g_all.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel_g_all.loc[ (afi_sel_g_all['NUM_SEC_MES'] == 1), '%_NUM_SECTOR'] = '1'

In [31]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==216]

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR
136,216,2009,9,641.34,PUB,1,1
137,216,2009,8,641.34,PUB,1,1
138,216,2009,7,641.34,PUB,1,1
139,216,2009,6,641.34,PUB,1,1
140,216,2009,5,641.34,PUB,1,1
...,...,...,...,...,...,...,...
59782937,216,2004,11,451.63,PRI;PUB,2,0.3735;0.6265
59782936,216,2004,10,451.63,PUB;PRI,2,0.6265;0.3735
59782935,216,2004,9,451.63,PUB;PRI,2,0.6265;0.3735
162,216,2004,8,282.95,PUB,1,1


In [32]:
#Se reinician los index
afi_sel_g_all.reset_index(inplace=True)
afi_sel_g_all.rename(columns={'index': 'nuevo_indice'}, inplace=True)
afi_sel_g_all.drop(columns=['nuevo_indice'], inplace=True)

In [33]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126]

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR
0,126,2010,12,8.06,PRI,1,1
1,126,2010,11,250.00,PRI,1,1
2,126,2010,10,250.00,PRI,1,1
3,126,2010,9,250.00,PRI,1,1
4,126,2010,8,250.00,PRI,1,1
...,...,...,...,...,...,...,...
68,126,2005,4,161.67,PRI,1,1
69,126,2005,3,161.67,PRI,1,1
70,126,2005,2,161.67,PRI,1,1
71,126,2005,1,161.67,PRI,1,1


In [34]:
#se crean los grupos de 12 meses trabajados para tener
n_grupo = 12
afi_sel_g_all['GRUPO'] = (afi_sel_g_all.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
#Se saca el salario promedio de cada grupo
afi_sel_g_all['SAL_PROM_GRUPO'] = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')

In [35]:
afi_sel_g_all

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO
0,126,2010,12,8.06,PRI,1,1,1,229.838333
1,126,2010,11,250.00,PRI,1,1,1,229.838333
2,126,2010,10,250.00,PRI,1,1,1,229.838333
3,126,2010,9,250.00,PRI,1,1,1,229.838333
4,126,2010,8,250.00,PRI,1,1,1,229.838333
...,...,...,...,...,...,...,...,...,...
62130162,23594465,2012,9,10528.81,PRI,1,1,6,11149.854545
62130163,23594465,2012,8,10722.25,PRI,1,1,6,11149.854545
62130164,23594465,2012,7,10722.25,PRI,1,1,6,11149.854545
62130165,23594465,2012,6,13046.07,PRI,1,1,6,11149.854545


In [36]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126]['GRUPO'].unique()

array([1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [37]:
#Se obtienen los sueldos promedios para cada grupo y cada cedula
inicio = time.time()
df1 = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].first().reset_index().copy()
df1 = df1.sort_values(by = ['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 6.51384162902832  segundos


In [38]:
df1

,CEDULA_COD,GRUPO,SAL_PROM_GRUPO
1,126,2,250.000000
2,126,3,241.666667
0,126,1,229.838333
3,126,4,170.000000
4,126,5,161.670000
...,...,...,...
5381435,23594465,3,17023.186667
5381434,23594465,2,16884.076667
5381433,23594465,1,16205.699167
5381436,23594465,4,15750.057500


In [39]:
 df1[df1['CEDULA_COD']==126]

,CEDULA_COD,GRUPO,SAL_PROM_GRUPO
1,126,2,250.000000
2,126,3,241.666667
0,126,1,229.838333
3,126,4,170.000000
4,126,5,161.670000
5,126,6,161.670000
6,126,7,147.450000


In [40]:
#Se seleccionan los 5 mejores años de sueldo
inicio = time.time()
top = 5
#top_sal = df1.groupby('CEDULA_COD')['SAL_PROM_GRUPO'].nlargest(top).reset_index(level=0, drop=True)
indices = df1.groupby('CEDULA_COD').apply(lambda x: x.index[:top]).explode()
df2 = df1.loc[indices].reset_index(drop=True)
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 9.637621641159058  segundos


In [41]:
df2[df2['CEDULA_COD']==126]

,CEDULA_COD,GRUPO,SAL_PROM_GRUPO
0,126,2,250.000000
1,126,3,241.666667
2,126,1,229.838333
3,126,4,170.000000
4,126,5,161.670000


In [42]:
#Seleccionar los cinco mejores años del grupo total de años de aporte
combinaciones = set(zip(df2['CEDULA_COD'], df2['GRUPO']))
afi_sel_g_all['GRUPO_SEL'] = [(cedula, grupo) in combinaciones for cedula, grupo in zip(afi_sel_g_all['CEDULA_COD'], afi_sel_g_all['GRUPO'])]
afi_sel_g_all['GRUPO_SEL'] = afi_sel_g_all['GRUPO_SEL'].astype(int)

In [43]:
#Verificar que todas las cédulas tengan 60 registros de los salarios
ced_counts1 = afi_sel_g_all.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()
ced_counts1[ced_counts1 < 60].index.nunique()

0

In [44]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126].tail(60)

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL
13,126,2009,11,250.00,PRI,1,1,2,250.000000,1
14,126,2009,10,250.00,PRI,1,1,2,250.000000,1
15,126,2009,9,250.00,PRI,1,1,2,250.000000,1
16,126,2009,8,250.00,PRI,1,1,2,250.000000,1
17,126,2009,7,250.00,PRI,1,1,2,250.000000,1
18,126,2009,6,250.00,PRI,1,1,2,250.000000,1
19,126,2009,5,250.00,PRI,1,1,2,250.000000,1
20,126,2009,4,250.00,PRI,1,1,2,250.000000,1
21,126,2009,3,250.00,PRI,1,1,2,250.000000,1
22,126,2009,2,250.00,PRI,1,1,2,250.000000,1


In [45]:
#Se crea una variable fecha para el período de mejores años y la base de cálculo
inicio = time.time()
mejores = afi_sel_g_all[afi_sel_g_all['GRUPO_SEL']==1].copy()
mejores.loc[:, 'FECHA'] = pd.to_datetime(mejores['ANIO'].astype(str) + '-' + mejores['MES'].astype(str).str.zfill(2) + '-01')

#Se calcula la base de calculo
df_prom = mejores.groupby('CEDULA_COD')['SALARIO'].mean()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 37.92071318626404  segundos


In [46]:
mejores

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,FECHA
0,126,2010,12,8.06,PRI,1,1,1,229.838333,1,2010-12-01
1,126,2010,11,250.00,PRI,1,1,1,229.838333,1,2010-11-01
2,126,2010,10,250.00,PRI,1,1,1,229.838333,1,2010-10-01
3,126,2010,9,250.00,PRI,1,1,1,229.838333,1,2010-09-01
4,126,2010,8,250.00,PRI,1,1,1,229.838333,1,2010-08-01
...,...,...,...,...,...,...,...,...,...,...,...
62130151,23594465,2013,8,12464.35,PRI,1,1,5,17158.475000,1,2013-08-01
62130152,23594465,2013,7,12451.36,PRI,1,1,5,17158.475000,1,2013-07-01
62130153,23594465,2013,6,12464.34,PRI,1,1,5,17158.475000,1,2013-06-01
62130154,23594465,2013,5,18476.35,PRI,1,1,5,17158.475000,1,2013-05-01


In [47]:
afi_sel_g_all['INI_CAL'] = afi_sel_g_all['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].max() )
afi_sel_g_all['FIN_CAL'] = afi_sel_g_all['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].min() )
afi_sel_g_all['BASE_CAL'] = afi_sel_g_all['CEDULA_COD'].map( df_prom )

In [48]:
caract['INI_CAL'] = caract['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].max() )
caract['FIN_CAL'] = caract['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].min() )
caract['BASE_CAL'] = caract['CEDULA_COD'].map( df_prom )
caract

,CEDULA_COD,INI_HL,FIN_HL,MES_AS,MES_TU,INI_CAL,FIN_CAL,BASE_CAL
0,126,2004-12-01,2010-12-01,73,73,2010-12-01,2006-01-01,210.635000
1,152,2004-07-01,2009-09-01,63,63,2009-09-01,2004-10-01,508.824667
2,216,2004-07-01,2009-09-01,98,63,2009-09-01,2004-10-01,566.339833
3,230,2004-12-01,2015-05-01,126,126,2015-05-01,2010-06-01,1285.000000
4,235,2004-08-01,2011-06-01,83,83,2011-06-01,2006-07-01,1325.717667
...,...,...,...,...,...,...,...,...
442565,23075971,2017-02-01,2022-10-01,66,66,2022-10-01,2017-11-01,399.333333
442566,23402574,2010-01-01,2019-02-01,110,110,2019-02-01,2013-03-01,424.344000
442567,23456125,2016-07-01,2022-03-01,69,69,2022-03-01,2016-07-01,579.554912
442568,23503686,2004-09-01,2011-11-01,87,87,2011-11-01,2006-12-01,215.797833


In [49]:
del df1, df2, mejores

In [50]:
afi_sel_g_all[ (afi_sel_g_all['CEDULA_COD']==126) & (afi_sel_g_all['GRUPO_SEL']==1)]['SALARIO'].mean()

210.63500000000002

In [51]:
afi_sel_g_all[ (afi_sel_g_all['CEDULA_COD']==126)]

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,INI_CAL,FIN_CAL,BASE_CAL
0,126,2010,12,8.06,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635
1,126,2010,11,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635
2,126,2010,10,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635
3,126,2010,9,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635
4,126,2010,8,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,126,2005,4,161.67,PRI,1,1,6,161.670000,0,2010-12-01,2006-01-01,210.635
69,126,2005,3,161.67,PRI,1,1,6,161.670000,0,2010-12-01,2006-01-01,210.635
70,126,2005,2,161.67,PRI,1,1,6,161.670000,0,2010-12-01,2006-01-01,210.635
71,126,2005,1,161.67,PRI,1,1,6,161.670000,0,2010-12-01,2006-01-01,210.635


In [52]:
# #     df = asignar_grupo(sect, n_grupo)
# #     df1 = valores_unicos_grupos(df)
# #     df2 = sel_top_salarios(df1, top)
# #     df = sel_grupo(df, df2)
#     df = fec_sel_grupo(df)
#     df = base_cal(df)
#     df = ati_base_cal_m1(df, inferior)
#     df = base_cal_sin_ati_m1(df)
#     df = sbu_ajuste_m1(df, SBU)

In [53]:
#Registros con años menores al 2000
afi_sel_g_all[afi_sel_g_all['FIN_CAL'].dt.year<2000]

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,INI_CAL,FIN_CAL,BASE_CAL
60607,10269,2010,5,733.03,PUB,1,1,1,688.09,1,2010-05-01,1999-02-01,524.108667
60608,10269,2010,4,733.03,PUB,1,1,1,688.09,1,2010-05-01,1999-02-01,524.108667
60609,10269,2010,3,733.03,PUB,1,1,1,688.09,1,2010-05-01,1999-02-01,524.108667
60610,10269,2010,2,733.03,PUB,1,1,1,688.09,1,2010-05-01,1999-02-01,524.108667
60611,10269,2010,1,733.03,PUB,1,1,1,688.09,1,2010-05-01,1999-02-01,524.108667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61166244,20041080,1997,5,56.65,PRI,1,1,7,56.65,0,2015-05-01,1998-07-01,114.486667
61166245,20041080,1997,4,56.65,PRI,1,1,7,56.65,0,2015-05-01,1998-07-01,114.486667
61166246,20041080,1997,3,56.65,PRI,1,1,7,56.65,0,2015-05-01,1998-07-01,114.486667
61166247,20041080,1997,2,56.65,PRI,1,1,7,56.65,0,2015-05-01,1998-07-01,114.486667


In [54]:
#Se agrega el valor del SBU
SBU = pd.DataFrame( { 'ANIO':[2000,2001,2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
                              2021, 2022, 2023, 2024],
                      'VALOR':[57,85.65,105, 122, 136, 150, 160, 170, 200, 218, 240, 264, 292, 318, 340, 354, 366, 375, 386, 394, 400, 
                              400, 425, 450, 460]})
SBU

,ANIO,VALOR
0,2000,57.00
1,2001,85.65
2,2002,105.00
3,2003,122.00
4,2004,136.00
5,2005,150.00
6,2006,160.00
7,2007,170.00
8,2008,200.00
9,2009,218.00


In [55]:
afi_sel_g_all['SBU'] = afi_sel_g_all['ANIO'].map( SBU.groupby('ANIO')['VALOR'].first())

In [56]:
afi_sel_g_all

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,INI_CAL,FIN_CAL,BASE_CAL,SBU
0,126,2010,12,8.06,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0
1,126,2010,11,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0
2,126,2010,10,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0
3,126,2010,9,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0
4,126,2010,8,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62130162,23594465,2012,9,10528.81,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0
62130163,23594465,2012,8,10722.25,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0
62130164,23594465,2012,7,10722.25,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0
62130165,23594465,2012,6,13046.07,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0


In [57]:
#Cédula que tienen su historia laboral igual al SBU
afi_sel_g_all['ID_SBU'] = afi_sel_g_all['SALARIO'] == afi_sel_g_all['SBU']
#Si tiene -1 es un valor que no es un SBU en ese año y mes
afi_sel_g_all['ID_SBU'] = afi_sel_g_all['ID_SBU'].astype(int)-1
afi_sel_g_all

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,INI_CAL,FIN_CAL,BASE_CAL,SBU,ID_SBU
0,126,2010,12,8.06,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
1,126,2010,11,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
2,126,2010,10,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
3,126,2010,9,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
4,126,2010,8,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62130162,23594465,2012,9,10528.81,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0,-1
62130163,23594465,2012,8,10722.25,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0,-1
62130164,23594465,2012,7,10722.25,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0,-1
62130165,23594465,2012,6,13046.07,PRI,1,1,6,11149.854545,0,2018-03-01,2013-04-01,16604.299,292.0,-1


In [58]:
#Verificar que todas las cédulas tengan 60 registros
ced_counts2 = afi_sel_g_all.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()
ced_counts2[ced_counts2 < 60].index.nunique()

0

In [59]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126].shape

(73, 15)

In [60]:
afi_sel_g_all[(afi_sel_g_all['CEDULA_COD']==126) & (afi_sel_g_all['GRUPO_SEL']==1)]

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,INI_CAL,FIN_CAL,BASE_CAL,SBU,ID_SBU
0,126,2010,12,8.06,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
1,126,2010,11,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
2,126,2010,10,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
3,126,2010,9,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
4,126,2010,8,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
5,126,2010,7,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
6,126,2010,6,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
7,126,2010,5,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
8,126,2010,4,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
9,126,2010,3,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1


In [61]:
objeto5 = afi_sel_g_all

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto5, archivo)

In [62]:
del objeto5

In [63]:
#Selección de cedulas con sus mejores años de sueldo y que su salario no sea un SBU
data_sin_ati = afi_sel_g_all[ (afi_sel_g_all['GRUPO_SEL']==1) & (afi_sel_g_all['ID_SBU']!=-1)]
data_ati = afi_sel_g_all[ (afi_sel_g_all['GRUPO_SEL']==1) & (afi_sel_g_all['ID_SBU']!=0)]

In [64]:
data_ati.shape #(21022610, 15)
data_sin_ati.shape #(5449506, 15)

(5449506, 15)

In [65]:
data_ati[data_ati['CEDULA_COD']==126][['CEDULA_COD','ANIO','MES','SALARIO']].shape

(46, 4)

In [66]:
data_ati[data_ati['CEDULA_COD']==126]

,CEDULA_COD,ANIO,MES,SALARIO,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,GRUPO_SEL,INI_CAL,FIN_CAL,BASE_CAL,SBU,ID_SBU
0,126,2010,12,8.06,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
1,126,2010,11,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
2,126,2010,10,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
3,126,2010,9,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
4,126,2010,8,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
5,126,2010,7,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
6,126,2010,6,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
7,126,2010,5,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
8,126,2010,4,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1
9,126,2010,3,250.00,PRI,1,1,1,229.838333,1,2010-12-01,2006-01-01,210.635,240.0,-1


In [67]:
objeto6 = data_ati
objeto7 = data_sin_ati

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto6, archivo)
    pickle.dump(objeto7, archivo)
    
del objeto6, objeto7

In [ ]:
#Cargar los archivos
directorio = r_ruta
nombre_archivo = 'viu_clean_afiliados.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

objetos = []
try:
    with open(ruta_archivo, 'rb') as archivo:
        while True:
            try:
                objetos.append(pickle.load(archivo))
            except EOFError:
                break
except Exception as e:
    print(f"Ocurrió un error al intentar cargar el archivo: {e}")

# # Verifica la cantidad de objetos y accede a ellos
# print(f"El archivo contiene {len(objetos)} objetos.")
# for i, obj in enumerate(objetos):
#     print(f"Objeto {i+1}: {obj}")

# # Acceder a un objeto específico
# if len(objetos) >= 3:
#     afi_sel_g_all = objetos[2]  # Cargar el tercer objeto
# else:
#     print("El archivo no contiene suficientes objetos.")

In [ ]:
#Análisis de atípicos
#Método de clustering jerárquico
data_l = objetos[4] # = afi_sel_g_all del objeto5

In [ ]:
del objetos

In [ ]:
data = data_l[ (data_l['GRUPO_SEL']==1) ]
data = data.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[ True, True, True] )

data_si = data[ data['SALARIO'] >= data['SBU']] #Para evitar las aportaciones a tiempo parcial
data_no = data[ data['SALARIO'] < data['SBU']] #Sueldos menores al SBU
data_si = data_si.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
data_si

In [ ]:
print(data.shape) #Tamaño (26472116, 15)
print(data_si.shape) #(26006294, 15)
print(data_no.shape)  #(458368, 15)

In [ ]:
objetos[4][objetos[4]['CEDULA_COD']==188869]

In [ ]:
data[data['CEDULA_COD']==188869]

In [ ]:
data_si['INDICE']=data_si.index
data_si

In [ ]:
#data_dic = data_si.groupby('CEDULA_COD')['SALARIO'].apply(list).to_dict()
data_dic = {}
cel = data_si['CEDULA_COD'].values
for cedula in cel:
    data_dic[cedula]={'SALARIO': data_si[data_si['CEDULA_COD']==cedula]['SALARIO'].values.tolist(),
                      'INDICE' : data_si[data_si['CEDULA_COD']==cedula]['INDICE'].values.tolist() }


# data_no_dic = data_no.groupby('CEDULA_COD')['SALARIO'].apply(list).to_dict()
# data_no_dic
#data_dic = data_si.groupby('CEDULA_COD')['SALARIO'].apply(lambda x: dict(x.reset_index().values)).to_dict()
#data_dic
# cedula_array = data_si['CEDULA_COD'].values
# salario_array = data_si['SALARIO'].values
# index_array = np.arange(len(data_si))

# # Crear un diccionario para almacenar los resultados
# result = {}

# # Agrupar por 'CEDULA_COD' y construir el diccionario
# for cedula in np.unique(cedula_array):
#     mask = cedula_array == cedula
#     result[cedula] = dict(zip(index_array[mask], salario_array[mask]))

# result

In [ ]:
data_si[data_si['CEDULA_COD']==126]['SALARIO'].values.tolist()

In [ ]:
#Algoritmo jerarquico
data_val_ati = {}

for cedula in data_dic:
    if( len( data_dic[ cedula ] ) > 1 ): #Para formar al menos un cluster
        aux = np.array( data_dic[ cedula ] ).reshape(-1, 1)
        Z = linkage( aux , method='single', metric='euclidean')

        num_clusters = 2  # Puedes ajustar este valor según tus necesidades
        clusters = fcluster(Z, num_clusters, criterion='maxclust')

        if( len( np.unique(clusters) ) > 1 ): #Para considerar al menos 2 cluster
            # Calcular el centroide de cada clúster
            cluster_centers = np.array([[ np.nanmean(aux[clusters == i], axis=0)[0], i ] for i in range(1, num_clusters + 1)] ) 
                    
            Q1 = np.quantile(aux, 0.25)
            Q3 = np.quantile(aux, 0.75)
            IQR = Q3-Q1
            LI = Q1 - 1.5 * IQR
            LS = Q3 + 1.5 * IQR

            cl_at = np.where( cluster_centers[:,0] > (LS +  1e-8) )[0]
            cluster_centers[cl_at][:, 1]

            mod_aux = np.zeros((len(aux), 2))
            mod_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
            mod_aux[np.isin(clusters, cluster_centers[cl_at][:, 1]), 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1
            mod_aux

            data_val_ati[cedula] = {'SALARIO': mod_aux[:, 0].tolist(),
                                    'ATIPICO': mod_aux[:, 1].tolist() }
        else:
            data_val_ati[cedula] = { 'SALARIO': aux.flatten().tolist(),
                                     'ATIPICO':  [-1] * len(aux) } 
            
    else:
        data_val_ati[cedula] = { 'SALARIO': aux.flatten().tolist(),
                                 'ATIPICO':  [-1] * len(aux) }


In [ ]:
data1 = { 'CEDULA': [], 'SALARIO': [], 'ATIPICO': [] }

# Llenar las listas con los datos del diccionario
for cedula, values in data_val_ati.items():
    salario = values['SALARIO']
    atipico = values['ATIPICO']
    num_rows = len(salario)
    
    # Extender las listas en el diccionario de datos
    data1['CEDULA'].extend([cedula] * num_rows)
    data1['SALARIO'].extend(salario)
    data1['ATIPICO'].extend(atipico)

# Convertir el diccionario en DataFrame
df = pd.DataFrame(data1)

In [ ]:
df

In [ ]:
data_dict = {
126: {'SALARIO': [161.67,161.67, 161.67],
      'ATIPICO': [0.0, 1,1]},
127: {'SALARIO': [161.67,161.67, 161.67],
      'ATIPICO': [0.0, 1,1]}
}

data_dict = {
    126: {'SALARIO': [161.67, 161.67, 161.67],
          'ATIPICO': [0.0, 1, 1]},
    127: {'SALARIO': [161.67, 161.67, 161.67],
          'ATIPICO': [0.0, 1, 1]}
}

# Crear listas para construir el DataFrame final
data = {
    'CEDULA': [],
    'SALARIO': [],
    'ATIPICO': []
}

# Llenar las listas con los datos del diccionario
for cedula, values in data_dict.items():
    salario = values['SALARIO']
    atipico = values['ATIPICO']
    num_rows = len(salario)
    
    # Extender las listas en el diccionario de datos
    data['CEDULA'].extend([cedula] * num_rows)
    data['SALARIO'].extend(salario)
    data['ATIPICO'].extend(atipico)

# Convertir el diccionario en DataFrame
df = pd.DataFrame(data)

# Mostrar el DataFrame resultante
print(df)


In [ ]:
len( data_dict[126]['SALARIO'])

In [ ]:
len( data_dict[126]['ATIPICO'] )

In [ ]:
cel_unico_cls = []
aux=None
aux = np.array( data_dic[126] ).reshape(-1, 1)

Z = linkage( aux , method='single', metric='euclidean')

num_clusters = 2  # Puedes ajustar este valor según tus necesidades
clusters = fcluster(Z, num_clusters, criterion='maxclust')

len( np.unique(clusters) ) 
cluster_centers = np.array([[ np.nanmean(aux[clusters == i], axis=0)[0], i ] for i in range(1, num_clusters + 1)] ) 
cluster_centers                  
Q1 = np.quantile(aux, 0.25)
Q3 = np.quantile(aux, 0.75)
IQR = Q3-Q1
LI = Q1 - 1.5 * IQR
LS = Q3 + 1.5 * IQR

cl_at = np.where( cluster_centers[:,0] > (LS +  1e-8) )[0]
cluster_centers[cl_at][:, 1]

mod_aux = np.zeros((len(aux), 2))
mod_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
mod_aux[np.isin(clusters, cluster_centers[cl_at][:, 1]), 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1
mod_aux

print(len(mod_aux))
len(mod_aux[:, 0].tolist())
len(mod_aux[:, 1].tolist())
# data_val_ati[cedula] = {'SALARIO': mod_aux[:, 0].tolist(),
#                                     'ATIPICO': mod_aux[:, 1].tolist() }

# mod_aux = np.zeros((len(aux), 2))
# mod_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
# mod_aux[np.isin(clusters, cluster_centers[cl_at][:, 1]), 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1
# mod_aux

# d={}

# d[216]={'SALARIO': mod_aux[:, 0].tolist(),
#         'ATIPICO': mod_aux[:, 1].tolist() }



In [ ]:
len(aux)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data_dic[2778], marker='o', linestyle='-', color='b', label='Salarios')  # Graficar como línea con puntos
plt.title('Distribución de Salarios')
plt.xlabel('Índice')
plt.ylabel('Salario')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Mostrar el gráfico
plt.show()


In [ ]:
len(a)

In [ ]:
aux

In [ ]:
import numpy as np

# Ejemplo de datos auxiliares y clusters
aux = np.array([
    [161.67], [161.67], [161.67],
    [170.], [170.], [200.], [200.], [250.]
])

clusters = np.array([
    2, 2, 2,
    1, 1, 1, 1, 1
], dtype=np.int32)

# Crear un array modificado según los clusters
modified_aux = np.zeros((len(aux), 2))
modified_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
modified_aux[clusters == 1, 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1

print("Array modificado:")
print(modified_aux)

In [ ]:
plt.figure(figsize=(25, 10))
plt.title('Dendrograma')
plt.xlabel('índice de la muestra')
plt.ylabel('distancia')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
Z

In [ ]:
#Código para graficar los datos
plt.figure(figsize=(25, 10))
plt.title('Dendrograma')
plt.xlabel('índice de la muestra')
plt.ylabel('distancia')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data_dic[2778], marker='o', linestyle='-', color='b', label='Salarios')  # Graficar como línea con puntos
plt.title('Distribución de Salarios')
plt.xlabel('Índice')
plt.ylabel('Salario')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Mostrar el gráfico
plt.show()